In [ ]:
import datacube
dc = datacube.Datacube(app="OpenDataCube_Testing_Load")

In [ ]:
#Tile query
xarr_tile=dc.load(product="LS8_OLI_LASRC", longitude=(-72, -71), latitude=(3,4), 
                  time=("2016-01-01","2016-02-29"), measurements=['blue','green','red'])
print(xarr_tile['blue'].shape)

#Piece of ingested image query
xarr_piece_ingested=dc.load(product="LS8_OLI_LASRC", longitude=(-71.81827, -70.87436), latitude=(2.849349, 3.797007), 
                            time=("2016-01-25 15:00:27","2016-01-25 15:00:27"), measurements=['blue','green','red'])
print(xarr_piece_ingested['blue'].shape)

#Tile query starting from .5
xarr_tile_5=dc.load(product="LS8_OLI_LASRC", longitude= (-71.5, -70.5), latitude=(2.5, 3.5), 
                    time=("2016-01-01","2016-02-29"), measurements=['blue','green','red'])
print(xarr_tile_5['blue'].shape)

#piece of ingested image starting from .5
xarr_piece_ingested_5=dc.load(product="LS8_OLI_LASRC", longitude= (-71.5, -70.5), latitude=(2.5, 3.5), 
                    time=("2016-01-25 15:00:27","2016-01-25 15:00:27"), measurements=['blue','green','red'])
print(xarr_piece_ingested_5['blue'].shape)



In [ ]:
from datacube.drivers.netcdf import writer as netcdf_writer
from datacube.utils.geometry import CRS
from datacube.utils import geometry, data_resolution_and_offset
import xarray as xr
import numpy as np
nodata=-9999
def saveNC(output,filename,history):
    nco=netcdf_writer.create_netcdf(filename)
    nco.history = (history.encode('ascii','replace'))

    coords=output.coords
    cnames=()
    for x in coords:
        if not 'units' in coords[x].attrs:
            if x == "time":
                coords[x].attrs["units"]=u"seconds since 1970-01-01 00:00:00"
        netcdf_writer.create_coordinate(nco, x, coords[x].values, coords[x].units)
        cnames=cnames+(x,)
    _crs=output.crs
    if isinstance(_crs, xr.DataArray):
        _crs=CRS(str(_crs.spatial_ref))
    netcdf_writer.create_grid_mapping_variable(nco, _crs)
    for band in output.data_vars:
        #Para evitar problemas con xarray <0.11
        if band in coords.keys() or band == 'crs':
            continue
        output.data_vars[band].values[np.isnan(output.data_vars[band].values)]=nodata
        var= netcdf_writer.create_variable(nco, band, netcdf_writer.Variable(output.data_vars[band].dtype, nodata, cnames, None) ,set_crs=True)
        var[:] = netcdf_writer.netcdfy_data(output.data_vars[band].values)
    nco.close()

In [ ]:
#saving querys
history = u'Creado con CDCOL y el Open Data Cube'
saveNC(xarr_tile,"/home/cubo/jupyter/odc_testing_query_tile.nc",history)
saveNC(xarr_piece_ingested,"/home/cubo/jupyter/odc_testing_query_piece_ingested.nc",history)
saveNC(xarr_tile_5,"/home/cubo/jupyter/odc_testing_query_tile_5.nc",history)
saveNC(xarr_piece_ingested_5,"/home/cubo/jupyter/odc_testing_query_piece_ingested_5.nc",history)